In [32]:
import numpy as np
import pandas as pd
import re
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
data = pd.read_csv('data/flipkart_com-ecommerce_sample.csv')
data.head()

,uniq_id,crawl_timestamp,product_url,product_name,product_category_tree,pid,retail_price,discounted_price,image,is_FK_Advantage_product,description,product_rating,overall_rating,brand,product_specifications
0,c2d766ca982eca8304150849735ffef9,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2FF9KEDEFGF,999.0,379.0,"[""http://img5a.flixcart.com/image/short/u/4/a/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
1,7f7036a6d550aaa89d34c77bd39a5e48,2016-03-25 22:59:23 +0000,http://www.flipkart.com/fabhomedecor-fabric-do...,FabHomeDecor Fabric Double Sofa Bed,"[""Furniture >> Living Room Furniture >> Sofa B...",SBEEH3QGU7MFYJFY,32157.0,22646.0,"[""http://img6a.flixcart.com/image/sofa-bed/j/f...",False,FabHomeDecor Fabric Double Sofa Bed (Finish Co...,No rating available,No rating available,FabHomeDecor,"{""product_specification""=>[{""key""=>""Installati..."
2,f449ec65dcbc041b6ae5e6a32717d01b,2016-03-25 22:59:23 +0000,http://www.flipkart.com/aw-bellies/p/itmeh4grg...,AW Bellies,"[""Footwear >> Women's Footwear >> Ballerinas >...",SHOEH4GRSUBJGZXE,999.0,499.0,"[""http://img5a.flixcart.com/image/shoe/7/z/z/r...",False,Key Features of AW Bellies Sandals Wedges Heel...,No rating available,No rating available,AW,"{""product_specification""=>[{""key""=>""Ideal For""..."
3,0973b37acd0c664e3de26e97e5571454,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2F6HUZMQ6SJ,699.0,267.0,"[""http://img5a.flixcart.com/image/short/6/2/h/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
4,bc940ea42ee6bef5ac7cea3fb5cfbee7,2016-03-25 22:59:23 +0000,http://www.flipkart.com/sicons-all-purpose-arn...,Sicons All Purpose Arnica Dog Shampoo,"[""Pet Supplies >> Grooming >> Skin & Coat Care...",PSOEH3ZYDMSYARJ5,220.0,210.0,"[""http://img5a.flixcart.com/image/pet-shampoo/...",False,Specifications of Sicons All Purpose Arnica Do...,No rating available,No rating available,Sicons,"{""product_specification""=>[{""key""=>""Pet Type"",..."


In [3]:
data.sample()

,uniq_id,crawl_timestamp,product_url,product_name,product_category_tree,pid,retail_price,discounted_price,image,is_FK_Advantage_product,description,product_rating,overall_rating,brand,product_specifications
19146,ae75ad2a6b3f8ea128b42b404cc06793,2016-03-02 19:09:22 +0000,http://www.flipkart.com/histeria-hand-held-bag...,Histeria Hand-held Bag,"[""Bags, Wallets & Belts >> Bags >> Hand Bags >...",HMBEGCVEZUBUWPTW,999.0,799.0,"[""http://img6a.flixcart.com/image/hand-messeng...",False,Key Features of Histeria Hand-held Bag Ideal F...,No rating available,No rating available,Histeria,"{""product_specification""=>[{""key""=>""Closure"", ..."


In [4]:
data.columns

Index(['uniq_id', 'crawl_timestamp', 'product_url', 'product_name',
       'product_category_tree', 'pid', 'retail_price', 'discounted_price',
       'image', 'is_FK_Advantage_product', 'description', 'product_rating',
       'overall_rating', 'brand', 'product_specifications'],
      dtype='object')

The history saving thread hit an unexpected error (OperationalError('database is locked')).History will not be written to the database.


In [5]:
data = data.drop(['pid', 'crawl_timestamp', 'product_url', 'retail_price', 'discounted_price',
       'image', 'is_FK_Advantage_product', 'product_rating','overall_rating','brand'],axis=1)

In [6]:
data.head()

,uniq_id,product_name,product_category_tree,description,product_specifications
0,c2d766ca982eca8304150849735ffef9,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",Key Features of Alisha Solid Women's Cycling S...,"{""product_specification""=>[{""key""=>""Number of ..."
1,7f7036a6d550aaa89d34c77bd39a5e48,FabHomeDecor Fabric Double Sofa Bed,"[""Furniture >> Living Room Furniture >> Sofa B...",FabHomeDecor Fabric Double Sofa Bed (Finish Co...,"{""product_specification""=>[{""key""=>""Installati..."
2,f449ec65dcbc041b6ae5e6a32717d01b,AW Bellies,"[""Footwear >> Women's Footwear >> Ballerinas >...",Key Features of AW Bellies Sandals Wedges Heel...,"{""product_specification""=>[{""key""=>""Ideal For""..."
3,0973b37acd0c664e3de26e97e5571454,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",Key Features of Alisha Solid Women's Cycling S...,"{""product_specification""=>[{""key""=>""Number of ..."
4,bc940ea42ee6bef5ac7cea3fb5cfbee7,Sicons All Purpose Arnica Dog Shampoo,"[""Pet Supplies >> Grooming >> Skin & Coat Care...",Specifications of Sicons All Purpose Arnica Do...,"{""product_specification""=>[{""key""=>""Pet Type"",..."


In [7]:
data['uniq_id'].nunique()

20000

In [8]:
data.shape

(20000, 5)

In [9]:
data.dtypes

uniq_id                   object
product_name              object
product_category_tree     object
description               object
product_specifications    object
dtype: object

In [10]:
data['product_category_tree'][3]

'["Clothing >> Women\'s Clothing >> Lingerie, Sleep & Swimwear >> Shorts >> Alisha Shorts >> Alisha Solid Women\'s Cycling Shorts"]'

# Removing unnecessary symbols from String

In [11]:
def repair(text):
    text = str(text)
    pattern = re.compile('[\([{})\]]')
    text= pattern.sub(r'',text)
    text = text.replace('"','')
    text = text.replace(',','')
    text = text.replace('&','')
    pattern = re.compile('>>')
    return pattern.sub(r'',text)

In [12]:
repair('["Clothing >> Women\'s Clothing >> Lingerie, Sleep & Swimwear >> Shorts >> Alisha Shorts >> Alisha Solid Women\'s Cycling Shorts"]')

"Clothing  Women's Clothing  Lingerie Sleep  Swimwear  Shorts  Alisha Shorts  Alisha Solid Women's Cycling Shorts"

In [13]:
data['product_category_tree'] = data['product_category_tree'].apply(repair)

In [14]:
data.sample(3)

,uniq_id,product_name,product_category_tree,description,product_specifications
17963,dee6f3907ba56288d1a32e84d4948e0e,Globus Solid Women's A-line Skirt,Clothing Women's Clothing Western Wear Dres...,Globus Solid Women's A-line Skirt - Buy Blue G...,"{""product_specification""=>[{""key""=>""Number of ..."
10119,2e4410f1739a5ec926cd8c47213e4d9b,Clovia Women's Tube Bra,Clothing Women's Clothing Lingerie Sleep Sw...,Clovia Women's Tube Bra\n ...,"{""product_specification""=>[{""key""=>""Brand Colo..."
413,038459e2a0372ee61a11401f4cc51bf2,"Soie Casual, Festive, Formal, Lounge Wear, Par...",Clothing Women's Clothing Western Wear Shir...,"Soie Casual, Festive, Formal, Lounge Wear, Par...","{""product_specification""=>[{""key""=>""Sleeve"", ""..."


In [15]:
data['description'][0]

"Key Features of Alisha Solid Women's Cycling Shorts Cotton Lycra Navy, Red, Navy,Specifications of Alisha Solid Women's Cycling Shorts Shorts Details Number of Contents in Sales Package Pack of 3 Fabric Cotton Lycra Type Cycling Shorts General Details Pattern Solid Ideal For Women's Fabric Care Gentle Machine Wash in Lukewarm Water, Do Not Bleach Additional Details Style Code ALTHT_3P_21 In the Box 3 shorts"

In [16]:
data['product_specifications'][0]

'{"product_specification"=>[{"key"=>"Number of Contents in Sales Package", "value"=>"Pack of 3"}, {"key"=>"Fabric", "value"=>"Cotton Lycra"}, {"key"=>"Type", "value"=>"Cycling Shorts"}, {"key"=>"Pattern", "value"=>"Solid"}, {"key"=>"Ideal For", "value"=>"Women\'s"}, {"value"=>"Gentle Machine Wash in Lukewarm Water, Do Not Bleach"}, {"key"=>"Style Code", "value"=>"ALTHT_3P_21"}, {"value"=>"3 shorts"}]}'

# Dropping 'specs' because equivalent to Description

In [17]:
data = data.drop('product_specifications',axis=1)

In [18]:
data.head(3)

,uniq_id,product_name,product_category_tree,description
0,c2d766ca982eca8304150849735ffef9,Alisha Solid Women's Cycling Shorts,Clothing Women's Clothing Lingerie Sleep Sw...,Key Features of Alisha Solid Women's Cycling S...
1,7f7036a6d550aaa89d34c77bd39a5e48,FabHomeDecor Fabric Double Sofa Bed,Furniture Living Room Furniture Sofa Beds F...,FabHomeDecor Fabric Double Sofa Bed (Finish Co...
2,f449ec65dcbc041b6ae5e6a32717d01b,AW Bellies,Footwear Women's Footwear Ballerinas AW Bel...,Key Features of AW Bellies Sandals Wedges Heel...


# Creating a new column

In [19]:
data['desc'] = data['product_category_tree']+data['description']

In [20]:
data['desc'][0]

"Clothing  Women's Clothing  Lingerie Sleep  Swimwear  Shorts  Alisha Shorts  Alisha Solid Women's Cycling ShortsKey Features of Alisha Solid Women's Cycling Shorts Cotton Lycra Navy, Red, Navy,Specifications of Alisha Solid Women's Cycling Shorts Shorts Details Number of Contents in Sales Package Pack of 3 Fabric Cotton Lycra Type Cycling Shorts General Details Pattern Solid Ideal For Women's Fabric Care Gentle Machine Wash in Lukewarm Water, Do Not Bleach Additional Details Style Code ALTHT_3P_21 In the Box 3 shorts"

In [21]:
data = data.drop(['product_category_tree','description'],axis=1)

In [22]:
data.head(3)

,uniq_id,product_name,desc
0,c2d766ca982eca8304150849735ffef9,Alisha Solid Women's Cycling Shorts,Clothing Women's Clothing Lingerie Sleep Sw...
1,7f7036a6d550aaa89d34c77bd39a5e48,FabHomeDecor Fabric Double Sofa Bed,Furniture Living Room Furniture Sofa Beds F...
2,f449ec65dcbc041b6ae5e6a32717d01b,AW Bellies,Footwear Women's Footwear Ballerinas AW Bel...


# Stemming : 
* Converting each word in string to its root form understandable universally

In [23]:
data['desc']= data['desc'].str.lower()

In [24]:
def stemming(text):
    text = str(text)
    stemmer = PorterStemmer()
    return " ".join([stemmer.stem(word) for word in text.split()])

In [25]:
data.dtypes

uniq_id         object
product_name    object
desc            object
dtype: object

In [26]:
stemming("Clothing  Women's Clothing  Lingerie Sleep  Swimwear  Shorts  Alisha Shorts  Alisha Solid Women's Cycling ShortsKey Features of Alisha Solid Women's Cycling Shorts Cotton Lycra Navy, Red, Navy,Specifications of Alisha Solid Women's Cycling Shorts Shorts Details Number of Contents in Sales Package Pack of 3 Fabric Cotton Lycra Type Cycling Shorts General Details Pattern Solid Ideal For Women's Fabric Care Gentle Machine Wash in Lukewarm Water, Do Not Bleach Additional Details Style Code ALTHT_3P_21 In the Box 3 shorts")

"cloth women' cloth lingeri sleep swimwear short alisha short alisha solid women' cycl shortskey featur of alisha solid women' cycl short cotton lycra navy, red, navy,specif of alisha solid women' cycl short short detail number of content in sale packag pack of 3 fabric cotton lycra type cycl short gener detail pattern solid ideal for women' fabric care gentl machin wash in lukewarm water, do not bleach addit detail style code altht_3p_21 in the box 3 short"

In [27]:
data['desc']= data['desc'].apply(stemming)

In [28]:
data.head(3)

,uniq_id,product_name,desc
0,c2d766ca982eca8304150849735ffef9,Alisha Solid Women's Cycling Shorts,cloth women' cloth lingeri sleep swimwear shor...
1,7f7036a6d550aaa89d34c77bd39a5e48,FabHomeDecor Fabric Double Sofa Bed,furnitur live room furnitur sofa bed futon fab...
2,f449ec65dcbc041b6ae5e6a32717d01b,AW Bellies,footwear women' footwear ballerina aw belliesk...


# Giving index to each uniq_id

In [29]:
for i in range(0,20000):
    data['uniq_id'][i] = i+1

# Function for recommending products

In [30]:
def recommend_products(product,similarity_model):
    product_index = data[data['product_name'] == product].index[0]
    distances = similarity_model[product_index]
    product_list = sorted(list(enumerate(distances)),reverse=True,key=lambda x: x[1])[1:6]
    output={}
    name_list = []
    sim_list = []
    for x in product_list:
        prod_name = data.iloc[x[0]].product_name
        name_list.append(prod_name)
        prod_sim = np.round((x[1]*100),2)
        sim_list.append(prod_sim)
        
    print('Checked Product :::::   ',product)    
    name_list=np.array(name_list)
    sim_list=np.array(sim_list)
    dat = np.reshape([[name_list],[sim_list]],(2,5))
    df = pd.DataFrame(dat.T,columns=['Recommended Product','Similarity(%age)'],index=[0,1,2,3,4])
    return df

# Bag Of Words Vectorizer

In [34]:
vect_bow = CountVectorizer(max_features=5000,stop_words='english')

In [37]:
fitter_bow = vect_bow.fit_transform(data['desc']).toarray()

In [39]:
vect_bow.get_feature_names()[4999]

C:\Users\91700\Desktop\recommend\env\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


'zyxel'

# Cosine similarity:
* gets similarity of each data item to all other 
* 20000 elements with all 20000

In [41]:
similarity_bow = cosine_similarity(fitter_bow)

In [42]:
similarity_bow[0]

array([1.        , 0.07413844, 0.16552118, ..., 0.01001453, 0.01001453,
       0.01001453])

In [43]:
np.shape(similarity_bow)

(20000, 20000)

# Enumerating similarity matrix

In [45]:
sorted(list(enumerate(similarity_bow[0])),reverse=True,key=lambda x: x[1])[1:6]

[(6, 0.99115475481814),
 (3, 0.9851141287811143),
 (9, 0.973455562767816),
 (13, 0.9704142011834319),
 (15, 0.9704142011834319)]

In [46]:
s = 'Sicons All Purpose Arnica Dog Shampoo'
recommend_products(s,similarity_bow)

Checked Product :::::    Sicons All Purpose Arnica Dog Shampoo


,Recommended Product,Similarity(%age)
0,Sicons All Purpose Tea Tree Dog Shampoo,77.57
1,Sicons Conditioning Conditoner Dog Shampoo,74.7
2,"Four Paws Sweats for Cat, Dog",37.61
3,Magideal Raincoat for Dog,35.44
4,Royal Canin Maxi Starter 1kg Vegetable Dog Food,32.69


# Similarity on a smaller Data
* 5000 data points only

In [48]:
data_small = data.iloc[:5000,:]

In [49]:
vect_bow = CountVectorizer(max_features=5000,stop_words='english')

In [50]:
fitter_bow = vect_bow.fit_transform(data_small['desc']).toarray()

In [51]:
vect_bow.get_feature_names()[4999]

C:\Users\91700\Desktop\recommend\env\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


'zyaden'

In [52]:
similarity_bow = cosine_similarity(fitter_bow)

In [53]:
similarity_bow[0]

array([1.        , 0.07454264, 0.16751548, ..., 0.        , 0.        ,
       0.        ])

In [54]:
np.shape(similarity_bow)

(5000, 5000)

In [55]:
sorted(list(enumerate(similarity_bow[0])),reverse=True,key=lambda x: x[1])[1:6]

[(3, 0.9969834018751053),
 (6, 0.9940832671600991),
 (9, 0.985184366143778),
 (13, 0.9821063603268448),
 (15, 0.9821063603268448)]

In [56]:
s = 'Sicons All Purpose Arnica Dog Shampoo'
recommend_products(s,similarity_bow)

Checked Product :::::    Sicons All Purpose Arnica Dog Shampoo


,Recommended Product,Similarity(%age)
0,Sicons All Purpose Tea Tree Dog Shampoo,75.07
1,Sicons Conditioning Conditoner Dog Shampoo,73.3
2,Loreal Revitalift essence,22.03
3,Dr. Thapar's Beard Wash With Aloevera,18.62
4,Nutrispa Age Defying day care,18.46


In [57]:
#import pickle
#pickle.dump(similarity_bow,open('similarity_bow.pkl','wb'))